# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Les Wang
- Sebastian Troncoso
- Tamuz Hod
- Josh Freilich
- Baizhou Xu

# Group Members IDs

- A12985002
- A14217903
- A15230380
- A13837230
- U08254404

# Research Question

How does the type of business affect the type of crime in the area?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


We predict that the more family friendly businesses in an area reduces the crimes relating to fighting and consume alcoholic beverages rate in the neighborhood whereas the less family friendly businesses will increase theis crime rate.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Data Cleaning

In this step the datasets are imported and a universal identifier (uid) is created from the stop_id and pid columns.

In [2]:
reason_df = pd.read_csv("Datasets/reason_for_stop.csv")
reason_df['uid'] = reason_df.stop_id.astype(str).str.cat(reason_df.pid.astype(str), sep='_')
details_df = pd.read_csv("Datasets/basic_details.csv",low_memory=False)
details_df['uid'] = details_df.stop_id.astype(str).str.cat(details_df.pid.astype(str), sep='_')


Next the datasets are merged on the <b>uid</b> and all columns that are not relevant to this analysis are dropped.

In [3]:
totalStop_df = pd.merge(reason_df, details_df, on='uid')
totalStop_df = totalStop_df.drop(["agency", "beat", "beat_name", "isschool","school_name", "isstudent"], axis=1)

###### This section will foucus on how we extracted the address information from this df and the use of the US cencus georecorder tool.


Step 1)
Get the address infromation from the data sets so it can be used in the US census geocoder tool.

The US census geocoder is accepts csv sheets of up to 10,000 lines where each line is formated in the follwing way:
"street number street name, City, State, " 
ex:
"4800 NIAGARA AVE, SAN DIEGO, CA,"

In [4]:
dfA_L = pd.read_csv('Data/tr_active1.csv')
dfL_Z = pd.read_csv('Data/tr_active2.csv')
df_business = pd.concat([dfA_L,dfL_Z])
df_business.head()

,BUSINESS ACCT#,DBA NAME,OWNERSHIP TYPE,ADDRESS,CITY,ZIP,STATE,BUSINESS PHONE,OWNER NAME,CREATION DT,START DT,EXP DT,NAICS,ACTIVITY DESC
0,1981012901,#1 FIFTH AVENUE,CORP,3845 05TH AVE,SAN DIEGO,92103-3140,CA,NaN,PUBWAY CORP,12/09/1981,12/09/1981,11/30/2019,722,FOOD SERVICES & DRINKING PLACES
1,2019005727,#WHO DO U THINK U ARE,SOLE,5754 ELK ST,SAN DIEGO,92114-4025,CA,(619) 362-0605,KIESHA MILLER,04/02/2019,04/08/2019,04/30/2020,5141,INFORMATION SERVICES
2,2010034224,% NATURAL GOMEZ,SOLE,4356 POPLAR ST,SAN DIEGO,92105-4657,CA,(619) 417-0105,JUANA GOMEZ,11/12/2010,11/12/2010,11/30/2019,446199,ALL OTHER HEALTH & PERSONAL CARE STORES
3,2015046247,& SALLY J FERNANDEZ,SOLE,17471 PLAZA CERADO SUITE 98,SAN DIEGO,92128-2285,CA,NaN,JAIME V FERNANDEZ,10/29/2015,01/01/2014,12/31/2019,541219,OTHER ACCOUNTING SERVICES
4,2007016185,+A PARTNERS,CORP,3916 NORMAL ST,SAN DIEGO,92103-3413,CA,(619) 294-4477,+A PARTNERS,07/11/2007,07/01/2004,06/30/2019,531,REAL ESTATE


We discovered that the CITY coulmn was "SAN DIEGO" even when address were located in "La Jolla" or "El Cajon"
So we tried each address for both of these city names as well 

In [5]:
def generateCSVadresses() :
    for j in range(0,18):
        f= open("business_loc"+str(j)+".csv","w+")
        print(j)
        for i in range(3333*j, 3333 * (j+1)):
            if(i >= len(df_business) -1):
                break
            row = df_business.iloc[i]
            try:
                f.write(str(i) + '001,' + row['ADDRESS'] + ',' + row['CITY'] + ',CA,\n')
                if(row['CITY'] == "SAN DIEGO") : 
                    f.write(str(i) + '002,' + row['ADDRESS'] + ',' + 'La Jolla' + ',CA,\n')
                    f.write(str(i) + '003,' + row['ADDRESS'] + ',' + 'El Cajon' + ',CA,\n')
            except:
                print("Error on line ", i)
        f.close()
# generateCSVadresses()

We then manuly inputed this into the georecorder (https://geocoding.geo.census.gov/geocoder/locations/addressbatch?form)
as this was to be done only once.

We than combinded the csv files using the command line and uploded the result to our data folder

In [6]:
# Input
data_file = "Data/GeoBusinessData/geoBusinessData.csv"

# Generate column names (will be 0, 1, 2, ..., largest_column_count - 1)
column_names = ['uid', 'address', 'match?', 'exact?', 'full_address', 'lon,lat', 'remove', 'LR']
geoBusiness_df = pd.read_csv(data_file, header=None, names=column_names)
geoBusiness_df.count()

uid             170370
address         170370
match?          170370
exact?           51353
full_address     51353
lon,lat          51353
remove           51353
LR               51353
dtype: int64

We proceded to drop any row that was not a match
and get back the orginal index location

In [7]:
geoBusiness_df = geoBusiness_df.dropna()
def shorten(uid) :
    return int(str(uid)[0:len(str(uid)) - 3])
geoBusiness_df['uid'] = geoBusiness_df['uid'].apply(shorten)

geoBusiness_df = geoBusiness_df.set_index('uid')
geoBusiness_df = geoBusiness_df.sort_index()
geoBusiness_df.head()

,address,match?,exact?,full_address,"lon,lat",remove,LR
uid,,,,,,,
1,"5754 ELK ST, SAN DIEGO, CA,",Match,Exact,"5754 ELK ST, SAN DIEGO, CA, 92114","-117.07374,32.704464",648577820.0,L
2,"4356 POPLAR ST, SAN DIEGO, CA,",Match,Exact,"4356 POPLAR ST, SAN DIEGO, CA, 92105","-117.10183,32.737766",195347398.0,L
3,"17471 PLAZA CERADO SUITE 98, SAN DIEGO, CA,",Match,Non_Exact,"17471 PLZ CERADO, SAN DIEGO, CA, 92128","-117.05527,33.028744",612877365.0,R
4,"3916 NORMAL ST, SAN DIEGO, CA,",Match,Exact,"3916 NORMAL ST, SAN DIEGO, CA, 92103","-117.14946,32.749218",195342882.0,L
5,"5383 AUBURN DR, SAN DIEGO, CA,",Match,Exact,"5383 AUBURN DR, SAN DIEGO, CA, 92105","-117.089806,32.73843",195350513.0,R


Since we got the addresses form a concated df of the bussines A_L and L_Z we had to match the index the foloing way:

In [8]:
dfA_L["lon"] = np.nan
dfA_L["lat"] = np.nan
dfL_Z["lon"] = np.nan
dfL_Z["lat"] = np.nan

In [9]:
for i in range (0,len(dfA_L)) :
    try :
        loc_lat = geoBusiness_df.loc[i,:]["lon,lat"]
        dfA_L.iat[i, len(dfA_L.columns) - 2] = loc_lat[0:loc_lat.find(',')]
        dfA_L.iat[i, len(dfA_L.columns) - 1] = loc_lat[loc_lat.find(',')+1:]
    except :
        continue

for i in range (0,len(dfL_Z)) :
    try :
        loc_lat = geoBusiness_df.loc[28302 + i,:]["lon,lat"]
        dfL_Z.iat[i, len(dfL_Z.columns) - 2] = loc_lat[0:loc_lat.find(',')]
        dfL_Z.iat[i, len(dfL_Z.columns) - 1] = loc_lat[loc_lat.find(',')+1:]
    except :
        continue

We than got rid of the bussinesses form whom we did not find an address and recombined them

In [10]:
dfA_L.dropna(inplace = True)
dfL_Z.dropna(inplace = True)

business_df = pd.concat([dfA_L, dfL_Z]).reset_index(drop=True)
business_df.head()

,BUSINESS ACCT#,DBA NAME,OWNERSHIP TYPE,ADDRESS,CITY,ZIP,STATE,BUSINESS PHONE,OWNER NAME,CREATION DT,START DT,EXP DT,NAICS,ACTIVITY DESC,lon,lat
0,2019005727,#WHO DO U THINK U ARE,SOLE,5754 ELK ST,SAN DIEGO,92114-4025,CA,(619) 362-0605,KIESHA MILLER,04/02/2019,04/08/2019,04/30/2020,5141,INFORMATION SERVICES,-117.073740,32.704464
1,2010034224,% NATURAL GOMEZ,SOLE,4356 POPLAR ST,SAN DIEGO,92105-4657,CA,(619) 417-0105,JUANA GOMEZ,11/12/2010,11/12/2010,11/30/2019,446199,ALL OTHER HEALTH & PERSONAL CARE STORES,-117.101830,32.737766
2,2007016185,+A PARTNERS,CORP,3916 NORMAL ST,SAN DIEGO,92103-3413,CA,(619) 294-4477,+A PARTNERS,07/11/2007,07/01/2004,06/30/2019,531,REAL ESTATE,-117.149460,32.749218
3,2016012911,1 ALL EYES AND EARS,SOLE,5383 AUBURN DR,SAN DIEGO,92105-3624,CA,(619) 395-1586,KATRINA GONZALEZ,09/26/2016,10/01/2016,09/30/2019,62419,OTHER INDIVIDUAL & FAMILY SERVICES,-117.089806,32.738430
4,2013047218,1 AMERICAN LIVESCAN,CORP,3690 MURPHY CANYON RD SUITE 102,SAN DIEGO,92123-4455,CA,(858) 874-3119,STONECREST GAS & WASH INC,04/30/2013,01/01/2012,12/31/2019,561611,INVESTIGATION SERVICES,-117.116520,32.809334


Finaly we got rid of everything that was not bussiness type and the location data

In [11]:
business_df = business_df[["ACTIVITY DESC", "lon", "lat"]]
business_df.head()

,ACTIVITY DESC,lon,lat
0,INFORMATION SERVICES,-117.073740,32.704464
1,ALL OTHER HEALTH & PERSONAL CARE STORES,-117.101830,32.737766
2,REAL ESTATE,-117.149460,32.749218
3,OTHER INDIVIDUAL & FAMILY SERVICES,-117.089806,32.738430
4,INVESTIGATION SERVICES,-117.116520,32.809334


The same process was done to the crime data

In [12]:
data_file = "Data/GeoCrimeData/reasonable_loc_result.csv"
column_names = ['uid', 'address', 'match?', 'exact?', 'full_address', 'lon,lat', 'remove', 'remove']
geoCrime_reasonable_df = pd.read_csv(data_file, header=None, names=column_names)

geoCrime_reasonable_df = geoCrime_reasonable_df.dropna()
geoCrime_reasonable_df['uid'] = geoCrime_reasonable_df['uid'].apply(shorten)

geoCrime_reasonable_df = geoCrime_reasonable_df.set_index('uid')
geoCrime_reasonable_df = geoCrime_reasonable_df.sort_index()

df_reasonable = geoCrime_reasonable_df

/opt/conda/lib/python3.6/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
for index, row in geoCrime_reasonable_df.iterrows() :
    loc_lat = row["lon,lat"]
    df_reasonable.at[index, "lon"] = loc_lat[0:loc_lat.find(',')]
    df_reasonable.at[index, "lat"] = loc_lat[loc_lat.find(',')+1:]

In [14]:
df_reasonable = df_reasonable[["lon", "lat"]]
df_reasonable = df_reasonable.dropna()
df_reasonable.head()

,lon,lat
uid,,
2443,-116.970985,32.785866
2443,-116.970985,32.785866
2447,-117.07028,32.75826
2449,-117.208534,32.873898
2451,-117.253426,32.793934


In [15]:
reasonable_crime_csv = df_reasonable.to_csv (r'reasonable_crime.csv', header=True)
business_df_csv = business_df.to_csv (r'business_df.csv', header=True)

We then combined our bussiness and crime data

In [17]:
business_df = pd.read_csv("business_df.csv", index_col=0)
business_df = business_df.loc[:, ~business_df.columns.str.contains('^Unnamed')]

reasonable_df = pd.read_csv("reasonable_crime.csv")
reasonable_df = reasonable_df.set_index("stop_id")

reasonable_df.drop_duplicates(keep="last",inplace=True)

alc_fight_df = reasonable_df.loc[reasonable_df["crime_type"].isin(["*fight / battery*", "*alcohol*"])]

Since the distances we are intrested in are all within a reletevly small geographic area we used the  Euclidean distance between tow points based on the understanding that in distanes of less than 5km the error will be minimal (~0.04%)
http://jonisalonen.com/2014/computing-distance-between-coordinates-can-be-simple-and-fast/

In [18]:
def getDistance(row,lat0,lon0) :
    deglen = 110.25
    x = row["lat"] - lat0
    y = (row["lon"] - lon0) * math.cos(lat0)
    return deglen * math.sqrt(x*x + y*y)
busi_alc_fight_df = business_df


This was calculation was time consuming so it was done in batches using the liberary's GIS lab "datacrancher" computer

In [19]:
'''
binNumber = 100
binSize = int(len(alc_fight_df)/binNumber)

for i in range(0,binNumber) :
    n=0
    print(i)
    for row in alc_fight_df.itertuples() :
        n=n+1
        if(n<i*binSize):
            continue 
        if(n>=(i+1)*binSize) :
            break
        try :
            busi_alc_fight_df[row[0]] = busi_alc_fight_df.apply(getDistanceTo_reasonable,axis=1,args=(row[3],row[2],))
        except :
            print(index1, "Did not work")
    busi_alc_fight_csv = busi_alc_fight_df.to_csv (r'busi_alc_fight_df3.csv', header=True) 
'''





'\nbinNumber = 100\nbinSize = int(len(alc_fight_df)/binNumber)\n\nfor i in range(0,binNumber) :\n    n=0\n    print(i)\n    for row in alc_fight_df.itertuples() :\n        n=n+1\n        if(n<i*binSize):\n            continue \n        if(n>=(i+1)*binSize) :\n            break\n        try :\n            busi_alc_fight_df[row[0]] = busi_alc_fight_df.apply(getDistanceTo_reasonable,axis=1,args=(row[3],row[2],))\n        except :\n            print(index1, "Did not work")\n    busi_alc_fight_csv = busi_alc_fight_df.to_csv (r\'busi_alc_fight_df3.csv\', header=True) \n'

The next step is to add the type of the crime to the column head

In [22]:
pre_procesising_df = pd.read_csv("business_alc_fight_df.csv", index_col=0)

reasonable_df = pd.read_csv("reasonable_crime.csv", engine = "python")
reasonable_df = reasonable_df.set_index("stop_id")
reasonable_df.drop_duplicates(keep="last",inplace=True) 
reasonable_df.rename_axis("ACTIVITY_DESC", inplace = True)



In [23]:
business_df = pre_procesising_df.copy(deep=True)
business_df.head()

business_df.set_index("ACTIVITY DESC", inplace = True)
business_df.drop("lat", axis=1, inplace = True)
business_df.drop("lon", axis=1, inplace = True)

In [26]:
def renameCrimeType(id) :
    if(str(id).isnumeric()):
        try :
            return id + "_" + alc_fight_df.loc[int(id)]["crime_type"][1:6]
        except :
            print(id, " Was not found")
    return id

business_df.rename(columns = lambda x: renameCrimeType(x), inplace = True)


2449  Was not found
2451  Was not found
2459  Was not found
2475  Was not found
2483  Was not found
2502  Was not found
2504  Was not found
2510  Was not found
2528  Was not found
2646  Was not found
2720  Was not found
2752  Was not found
2778  Was not found
2782  Was not found
2924  Was not found
2982  Was not found
2999  Was not found
3053  Was not found
3065  Was not found
3098  Was not found
3119  Was not found
3122  Was not found
3199  Was not found
3358  Was not found
3361  Was not found
3365  Was not found
3370  Was not found
3384  Was not found
3393  Was not found
3416  Was not found
3455  Was not found
3530  Was not found
3765  Was not found
3773  Was not found
3783  Was not found
3846  Was not found
3963  Was not found
3990  Was not found
4001  Was not found
4076  Was not found
4098  Was not found
4104  Was not found
4130  Was not found
4270  Was not found
4276  Was not found
4277  Was not found
4357  Was not found
4368  Was not found
4408  Was not found
4436  Was not found


In [27]:
business_df.head()

,2447_fight,2449,2451,2455_fight,2459,2475,2483,2485_alcoh,2502,2504,...,42876_fight,42877_fight,42905_fight,42943_fight,42952_alcoh,43034_alcoh,43078_alcoh,43138_alcoh,43179_alcoh,43180_fight
ACTIVITY DESC,,,,,,,,,,,,,,,,,,,,,
INFORMATION SERVICES,5.931638,18.754879,10.569127,10.826005,3.563687,10.839486,10.545842,12.152264,2.731830,2.731830,...,6.631222,2.684126,3.139930,2.130846,6.838503,2.307581,10.708582,1.763786,3.855569,2.378067
ALL OTHER HEALTH & PERSONAL CARE STORES,2.392861,15.066880,6.971492,7.206127,5.277289,7.231852,6.955788,8.598698,3.270481,3.270481,...,3.649836,3.130361,2.807738,4.033039,3.104694,4.103705,7.032878,4.122998,0.760198,2.700259
REAL ESTATE,2.214189,13.765513,5.396970,5.652816,5.976163,5.670268,5.376207,7.630572,4.044222,4.044222,...,2.080185,3.929264,3.378673,5.098842,1.899324,5.107794,5.535838,5.332971,1.494110,3.690369
OTHER INDIVIDUAL & FAMILY SERVICES,2.239962,15.007879,7.027915,7.251192,5.532868,7.281533,7.015202,8.483424,3.533317,3.533317,...,3.897728,3.391489,3.104835,4.231200,3.097075,4.314055,7.052160,4.286276,1.096052,2.936775
INVESTIGATION SERVICES,5.748061,7.209228,3.353508,3.089895,12.684715,3.215225,3.426751,2.089211,10.738622,10.738622,...,6.896470,10.619667,10.075759,11.749799,4.835224,11.776320,2.413282,11.930609,8.024520,10.340160


# Data Analysis & Results

One approch we used is assume bussiness have a gravity like force that couses crime to happen around them. We than tried to find the magnatiud of this force for each business type.

We started by transforming each distance to its inverse squred

In [25]:
weights_df = (1 / business_df) * (1 / business_df)

In [28]:
weights_df.head()

,2447_fight,2449,2451,2455_fight,2459,2475,2483,2485_alcoh,2502,2504,...,42876_fight,42877_fight,42905_fight,42943_fight,42952_alcoh,43034_alcoh,43078_alcoh,43138_alcoh,43179_alcoh,43180_fight
ACTIVITY DESC,,,,,,,,,,,,,,,,,,,,,
INFORMATION SERVICES,0.028422,0.002843,0.008952,0.008532,0.078741,0.008511,0.008992,0.006772,0.133996,0.133996,...,0.022741,0.138802,0.101429,0.220240,0.021383,0.187796,0.008720,0.321446,0.067270,0.176828
ALL OTHER HEALTH & PERSONAL CARE STORES,0.174649,0.004405,0.020575,0.019257,0.035907,0.019121,0.020668,0.013525,0.093493,0.093493,...,0.075068,0.102050,0.126849,0.061480,0.103744,0.059381,0.020218,0.058827,1.730402,0.137148
REAL ESTATE,0.203972,0.005277,0.034332,0.031295,0.028000,0.031102,0.034598,0.017175,0.061141,0.061141,...,0.231098,0.064771,0.087601,0.038464,0.277206,0.038330,0.032631,0.035161,0.447955,0.073428
OTHER INDIVIDUAL & FAMILY SERVICES,0.199305,0.004440,0.020246,0.019019,0.032666,0.018861,0.020320,0.013895,0.080100,0.080100,...,0.065823,0.086940,0.103734,0.055856,0.104255,0.053731,0.020107,0.054430,0.832411,0.115947
INVESTIGATION SERVICES,0.030266,0.019241,0.088920,0.104740,0.006215,0.096734,0.085160,0.229106,0.008672,0.008672,...,0.021025,0.008867,0.009850,0.007243,0.042773,0.007211,0.171705,0.007025,0.015530,0.009353


We decided to divide our data by crime type to see if this force acts diffrently on alcoholic crimes vs fighting

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*